# TP1- Revision sobre los SGBD relacionales

Para revisar algunos conceptos de los SGBD relacionales (en particular el modelo relacional y el lenguaje SQL), vamos a crear una pequeña Base de Datos relacional desde un programa en Python y a partir de datos que se encuentran en varios archivos textos.

La base de datos que queremos construir está representada por el modelo Entidad-Asociación siguiente:

<img src="modeloEA-1.png">

## 1. Transformación del Modelo Entidad-Asociación hacia Modelo Relacional

Proponer un modelo relacional que corresponde al modelo Entidad-Asociación.

Medios Prensa(PK Nombre, Dueño, Numero followers)
Noticias(PK ID, Titulo, Texto, fecha, #Nombre)

## 2. Crear la estructura de la base de datos desde un programa Python

### 2.1 Conectarse a MySQL desde Python

In [1]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1')
cursor = cnx.cursor()
try:
    cursor.execute('CREATE DATABASE tallerdb1')
except Exception as db_creation:
    print(db_creation)
cnx.close()

### 2.2 Crear las tablas

In [2]:
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
query = ("CREATE TABLE `medio_prensa` ("
        "  `nombre` varchar(100) NOT NULL,"
        "  `dueno` varchar(100) NOT NULL,"
        "  `numero_followers` int(11) NOT NULL DEFAULT 0,"
        "  PRIMARY KEY (`nombre`))")
try:
    cursor.execute(query)
except Exception as tbl_creation:
    print(tbl_creation)
query = ("CREATE TABLE `noticia` ("
        "  `id` int(11) NOT NULL AUTO_INCREMENT,"
        "  `titulo` varchar(255) NOT NULL,"
        "  `texto` text NOT NULL,"
        "  `fecha` datetime NOT NULL,"
        "  `nombre` varchar(100) NOT NULL," 
        "  PRIMARY KEY (`id`),"
        "  FOREIGN KEY (`nombre`) REFERENCES medio_prensa(`nombre`))")
try:
    cursor.execute(query)
except Exception as tbl_creation:
    print(tbl_creation)

# 3. Insertar los datos en la base de datos

## 3.1 Leer los archivos textos

In [163]:
noticias_f = open('noticias.txt', 'r')
noticias=[]
noticias = noticias_f.read().split('----------\n')
noticias_f.close()

medios_f = open('medios.csv','r')
medios=[]
medios = medios_f.read().split('\n')
medios_f.close()
    

['2017-02-28 23:09:06\n', '24HorasTVN\n', 'Casa misteriosa de Puerto Montt se convierte en atracción turística - Casa misteriosa de Puerto Montt se convierte en atracción turística - Redes Sociales - 24horas\n', 'Masiva concurrencia a la misteriosa casa de Puerto Montt donde se registrarían supuestos sucesos paranormales. Masiva concurrencia a la misteriosa casa de Puerto Montt donde se registrarían supuestos sucesos paranormales.\n', '----------\n', '2017-02-09 18:30:02\n']


### 3.2 Leer los archivos textos y insertar los datos

In [3]:
medios_f = open('medios.csv','r')
medios=[]
medios = medios_f.read().split('\n')
del medios[0]
medios_f.close()
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
query=''

for m in medios:
    medio=m.split(',')
    if len(medio)==3:
        query=("INSERT INTO medio_prensa (nombre, dueno, numero_followers)"
               "VALUES ('"+medio[0]+"','"+ medio[1] +"','"+ medio[2] +"');")
    else:
        for i in range(2,len(medio)-1):
            medio[1]+=','+medio[i]
        query=("INSERT INTO medio_prensa (nombre, dueno, numero_followers)"
                "VALUES ('"+medio[0]+"','"+ medio[1] +"','"+ medio[len(medio)-1] +"');")
    try:
        cursor.execute(query)
        #print(query)
    except Exception as insertFail_m:
        print(insertFail_m)
noticias_f = open('noticias.txt', 'r')
noticias=[]
noticias = noticias_f.read().split('----------\n')
noticias_f.close()
del noticias[len(noticias)-1]

for n in noticias:
    noticia=n.split('\n')
    query=("INSERT INTO noticia (fecha, nombre, titulo, texto)"
           "VALUES ('"+noticia[0]+"','"+ noticia[1] +"','"+ noticia[2] +"','"+ noticia[3] +"');")
    
    try:
        cursor.execute(query)
        #print(query)
    except Exception as insertFail_n:
        print(query,insertFail_n)
cnx.commit()
cursor.close()
cnx.close()



# 4. Consultar la base de datos

Listar los medios de prensa en Chile ordenado según su número de followers en Twitter.

In [4]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
cursor.execute('SELECT * FROM medio_prensa ORDER BY numero_followers;')
cnx.close()

In [6]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
    cursor.execute('SELECT COUNT(*) as Dueños FROM (SELECT dueno FROM medio_prensa GROUP BY dueno) as meh ;')
cnx.close()

Contar cuantes medios de prensa posee cada dueño.

In [7]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
cursor.execute('SELECT dueno as Dueño, count(*) as Numero FROM medio_prensa GROUP BY dueno;')
cnx.close()

Mostrar los últimos 10 títulos de noticias del medio "La Tercera"

In [6]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
cursor.execute('SELECT fecha FROM noticia WHERE nombre LIKE "%la tercera%" OR nombre LIKE "%latercera%" ORDER BY fecha DESC LIMIT 10;')
cnx.close()

Mostrar los nombre de medios y los títulos en los cuales aparacen la palabra "Valdivia".

In [12]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
cursor.execute('SELECT nombre, titulo FROM noticia WHERE nombre LIKE "%valdivia%" OR titulo LIKE "%valdivia%";')
cnx.close()

Ellipsis

Por cada medio, mostrar el número de títulos en los cuales aparecen la palabra "Puerto Montt".

In [14]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root', host='127.0.0.1', database='tallerdb1')
cursor = cnx.cursor()
cursor.execute('SELECT nombre, count(*) FROM noticia WHERE titulo LIKE "%puerto montt%" OR titulo LIKE "%puertomontt%" GROUP BY nombre;')
cnx.close()

Ellipsis